config.py 에서는 carblog analyzer 를 import 할 수 있도록 path 를 추가하며, soynlp 가 설치되어 있는지 확인합니다. 만약 soynlp 가 설치되지 않았다면 설치법을 error message 로 출력합니다.

In [1]:
import config

Available soynlp == 0.0.492
Available carblog_analyzer == 0.0.1
Available carblog_dataset


L parts 중에서 단어스럽지 않은 subwords 를 제거하기 위해 droprate 를 이용하여 word scoring 을 합니다.

In [2]:
from carblog_analyzer.tokenizer import word_score_by_droprate
from carblog_dataset import load_text

texts = load_text(category=10) # 소나타
word_scores = word_score_by_droprate(texts)

소나타 블로그에서 학습된 subwords 의 word score 입니다. '엔진'과 '엔진오일'은 단어 점수가 높지만, '엔진오'는 단어 점수가 낮습니다.

In [25]:
for word in '소 소나 소나타 엔진 엔진오 엔진오일 잔류제거 클래식'.split():
    print('{}\t: {}'.format(word, word_scores.get(word, 0)))

소	: 0
소나	: 0.012827566134373525
소나타	: 0.9532185399984898
엔진	: 0.6726685824508916
엔진오	: 0.002477390802385959
엔진오일	: 0.9162346586936079
잔류제거	: 0.8985024958402662
클래식	: 0.9366240097501524


Subword tokenizer 는 L parts 의 모든 subwords 중에서 dictionary 에 존재하는 단어만 선택하는 토크나이저입니다. 한 어절에서 overlap 되는 subwords 들이 return 됩니다.

In [81]:
from carblog_analyzer.tokenizer import SubwordTokenizer

subword_tokenizer = SubwordTokenizer({'엔진', '엔진오일', '소나타XG', '소나타'})
subword_tokenizer.tokenize('소나타XG의 엔진오일을 교체하는 예시입니다')

['소나타', '소나타XG', '엔진', '엔진오일']

모든 카테고리에서 최소 빈도수가 50 이상이며, droprate 가 0.1 보다 큰 subwords 를 학습하는 함수를 작성하여 모든 카테고리에 적용할 수 있는 subword to idx 를 학습합니다. 시간이 오래 걸리며, 약 4 GB 의 메모리를 필요로 합니다.

In [27]:
from carblog_dataset import load_text, load_category_index

def train_vocabulary_index(min_count=50, min_droprate_wordscore=0.1, verbose=False):
    idx_to_category = load_category_index()
    num_categories = len(idx_to_category)

    subword_dictionary = set()
    for c in range(num_categories):
        texts = load_text(category=c)
        word_scores = word_score_by_droprate(texts, min_count=min_count)
        word_scores = {w:s for w,s in word_scores.items() if s >= min_droprate_wordscore}
        subword_dictionary.update(word_scores)

        if verbose:
            args = (len(word_scores), idx_to_category[c], len(subword_dictionary))
            print('Extract {} subwords from [{}] category. Num of all subwords = {}'.format(*args))

    idx_to_subword = sorted(subword_dictionary)
    subword_to_idx = {sub:idx for idx, sub in enumerate(idx_to_subword)}
    return idx_to_subword, subword_to_idx

idx_to_subword, subword_to_idx = train_vocabulary_index(verbose=True)

Extract 36085 subwords from [A6] category. Num of all subwords = 36085
Extract 55982 subwords from [BMW5] category. Num of all subwords = 66434
Extract 48823 subwords from [BMW] category. Num of all subwords = 73351
Extract 33713 subwords from [K3] category. Num of all subwords = 79115
Extract 65179 subwords from [K5] category. Num of all subwords = 97831
Extract 47256 subwords from [K7] category. Num of all subwords = 100850
Extract 5905 subwords from [QM3] category. Num of all subwords = 100958
Extract 51284 subwords from [그랜저] category. Num of all subwords = 109962
Extract 18030 subwords from [벤츠E] category. Num of all subwords = 110284
Extract 16468 subwords from [산타페] category. Num of all subwords = 111473
Extract 61942 subwords from [소나타] category. Num of all subwords = 123194
Extract 34413 subwords from [스포티지] category. Num of all subwords = 124146
Extract 34932 subwords from [싼타페] category. Num of all subwords = 125221
Extract 32279 subwords from [쏘나타] category. Num of all subw

Subwords 를 학습하였으니, 이를 이용하여 각 카테고리에서 subword 가 등장한 document frequency 를 계산합니다. 이 부분도 시간이 오래 걸리니 각 단계별로 진행 상황을 출력합니다.

In [29]:
from collections import defaultdict
import numpy as np


def compute_document_frequency(documents, subword_to_idx):
    n_docs = len(documents)
    n_subwords = len(subword_to_idx)
    tokenizer = SubwordTokenizer(subword_to_idx)

    # count subwords
    df = defaultdict(int)
    for doc in documents:
        subword_set = set(tokenizer.tokenize(doc))
        for subword in subword_set:
            df[subword] += 1
    df = {sub:df_s/n_docs for sub, df_s in df.items()}

    # convert dict to as numpy.ndarray
    df_ratio = np.zeros(n_subwords)
    for sub, ratio in df.items():
        df_ratio[subword_to_idx[sub]] = ratio
    return df_ratio

def subword_df_distribution(subword_to_idx, verbose=False):
    idx_to_category = load_category_index()
    num_categories = len(idx_to_category)

    df_ratios = []
    for c in range(num_categories):
        texts = load_text(category=c)
        df_ratios.append(compute_document_frequency(texts, subword_to_idx))

        if verbose:
            print('Train DF from [{}] category'.format(idx_to_category[c]))
    df_ratios = np.vstack(df_ratios)
    return df_ratios

df_ratios = subword_df_distribution(subword_to_idx, verbose=True)

Train DF from [A6] category
Train DF from [BMW5] category
Train DF from [BMW] category
Train DF from [K3] category
Train DF from [K5] category
Train DF from [K7] category
Train DF from [QM3] category
Train DF from [그랜저] category
Train DF from [벤츠E] category
Train DF from [산타페] category
Train DF from [소나타] category
Train DF from [스포티지] category
Train DF from [싼타페] category
Train DF from [쏘나타] category
Train DF from [쏘렌토] category
Train DF from [아반떼] category
Train DF from [아반테] category
Train DF from [제네시스] category
Train DF from [코란도C] category
Train DF from [투싼] category
Train DF from [티구안] category
Train DF from [티볼리] category
Train DF from [파사트] category
Train DF from [폭스바겐골프] category
Train DF from [현기차] category
Train DF from [현대자동차] category
Train DF from [현대차] category


subword 를 입력하면 각 카테고리별로 subword 가 등장한 문서의 비율을 출력하는 함수를 만듭니다. percentage 와 카테고리 이름을 출력하고, 그 앞에 손쉽게 각 카테고리 별 비율을 비교할 수 있도록 bar 도 출력합니다.

In [82]:
idx_to_category = load_category_index()

def show_df_distribution(subword):
    def bar_str(df_i):
        bar_len = int(20 * df_i)
        return '#' * bar_len + '-' * (20 - bar_len)

    subword_idx = subword_to_idx.get(subword, -1)
    if subword_idx == -1:
        return
    df = df_ratios[:,subword_idx].reshape(-1)
    bars = df / df.max()

    print('Subword = {}'.format(subword))
    for i, (df_i, bar_i) in enumerate(zip(df, bars)):
        category = idx_to_category[i]
        perc = '%.3f' % (100 * df_i)
        bar = bar_str(bar_i)
        print('[{}]: ({} %)\t {}'.format(bar, perc, category))

클래식 장르 중 하나인 소나타 관련 문서에는 '악장' 뿐 아니라 음악 관련 단어들이 자주 등장합니다.

In [83]:
show_df_distribution(subword = '악장')

Subword = 악장
[--------------------]: (0.008 %)	 A6
[--------------------]: (0.009 %)	 BMW5
[--------------------]: (0.005 %)	 BMW
[--------------------]: (0.010 %)	 K3
[--------------------]: (0.015 %)	 K5
[--------------------]: (0.004 %)	 K7
[--------------------]: (0.000 %)	 QM3
[--------------------]: (0.004 %)	 그랜저
[--------------------]: (0.017 %)	 벤츠E
[--------------------]: (0.034 %)	 산타페
[####################]: (2.729 %)	 소나타
[--------------------]: (0.001 %)	 스포티지
[--------------------]: (0.001 %)	 싼타페
[--------------------]: (0.015 %)	 쏘나타
[--------------------]: (0.002 %)	 쏘렌토
[--------------------]: (0.002 %)	 아반떼
[--------------------]: (0.004 %)	 아반테
[--------------------]: (0.003 %)	 제네시스
[--------------------]: (0.000 %)	 코란도C
[--------------------]: (0.002 %)	 투싼
[--------------------]: (0.005 %)	 티구안
[--------------------]: (0.018 %)	 티볼리
[--------------------]: (0.015 %)	 파사트
[--------------------]: (0.002 %)	 폭스바겐골프
[--------------------]: (0.000 %)	 현기차
[---------

영화 '터미네이터4'의 부제는 '제네시스'입니다.

In [84]:
show_df_distribution(subword = '터미네이터')

Subword = 터미네이터
[--------------------]: (0.022 %)	 A6
[--------------------]: (0.051 %)	 BMW5
[--------------------]: (0.038 %)	 BMW
[--------------------]: (0.013 %)	 K3
[--------------------]: (0.015 %)	 K5
[--------------------]: (0.012 %)	 K7
[--------------------]: (0.008 %)	 QM3
[--------------------]: (0.011 %)	 그랜저
[--------------------]: (0.039 %)	 벤츠E
[--------------------]: (0.028 %)	 산타페
[--------------------]: (0.027 %)	 소나타
[--------------------]: (0.012 %)	 스포티지
[--------------------]: (0.011 %)	 싼타페
[--------------------]: (0.009 %)	 쏘나타
[--------------------]: (0.010 %)	 쏘렌토
[--------------------]: (0.011 %)	 아반떼
[--------------------]: (0.000 %)	 아반테
[####################]: (1.775 %)	 제네시스
[--------------------]: (0.008 %)	 코란도C
[--------------------]: (0.011 %)	 투싼
[--------------------]: (0.015 %)	 티구안
[--------------------]: (0.018 %)	 티볼리
[--------------------]: (0.000 %)	 파사트
[--------------------]: (0.008 %)	 폭스바겐골프
[--------------------]: (0.067 %)	 현기차
[------

코펜하겐에는 소설 '인어공주' 와 관련된 '티볼리' 공원이 있습니다. 이와 관련된 포스트도 상당히 많습니다.

In [85]:
show_df_distribution(subword = '인어공주')

Subword = 인어공주
[--------------------]: (0.019 %)	 A6
[--------------------]: (0.035 %)	 BMW5
[--------------------]: (0.030 %)	 BMW
[--------------------]: (0.022 %)	 K3
[--------------------]: (0.013 %)	 K5
[--------------------]: (0.014 %)	 K7
[--------------------]: (0.000 %)	 QM3
[--------------------]: (0.006 %)	 그랜저
[--------------------]: (0.019 %)	 벤츠E
[--------------------]: (0.028 %)	 산타페
[##------------------]: (0.123 %)	 소나타
[--------------------]: (0.002 %)	 스포티지
[--------------------]: (0.003 %)	 싼타페
[--------------------]: (0.009 %)	 쏘나타
[--------------------]: (0.012 %)	 쏘렌토
[--------------------]: (0.007 %)	 아반떼
[--------------------]: (0.000 %)	 아반테
[--------------------]: (0.015 %)	 제네시스
[--------------------]: (0.000 %)	 코란도C
[--------------------]: (0.001 %)	 투싼
[--------------------]: (0.005 %)	 티구안
[####################]: (0.851 %)	 티볼리
[--------------------]: (0.010 %)	 파사트
[--------------------]: (0.006 %)	 폭스바겐골프
[--------------------]: (0.000 %)	 현기차
[-------

'엔진오일'은 자동차 용어이기 떄문에 모든 카테고리에서 비슷한 비율로 등장합니다.

In [86]:
show_df_distribution(subword = '엔진오일')

Subword = 엔진오일
[###########---------]: (7.238 %)	 A6
[############--------]: (7.482 %)	 BMW5
[#######-------------]: (4.472 %)	 BMW
[################----]: (10.050 %)	 K3
[###############-----]: (9.150 %)	 K5
[###############-----]: (9.414 %)	 K7
[#######-------------]: (4.713 %)	 QM3
[#############-------]: (8.464 %)	 그랜저
[################----]: (9.963 %)	 벤츠E
[########------------]: (5.414 %)	 산타페
[########------------]: (5.403 %)	 소나타
[###################-]: (11.675 %)	 스포티지
[###############-----]: (9.633 %)	 싼타페
[################----]: (9.750 %)	 쏘나타
[################----]: (10.221 %)	 쏘렌토
[##############------]: (8.681 %)	 아반떼
[#############-------]: (8.065 %)	 아반테
[############--------]: (7.305 %)	 제네시스
[####################]: (12.148 %)	 코란도C
[###################-]: (11.667 %)	 투싼
[#########-----------]: (5.770 %)	 티구안
[###-----------------]: (2.246 %)	 티볼리
[###############-----]: (9.183 %)	 파사트
[##############------]: (8.666 %)	 폭스바겐골프
[#######-------------]: (4.486 %)	 현기차
[##

'티볼리'는 루이비통의 제품 이름 중 하나입니다.

In [87]:
show_df_distribution(subword = '루이비통')

Subword = 루이비통
[#-------------------]: (0.182 %)	 A6
[#-------------------]: (0.206 %)	 BMW5
[#-------------------]: (0.265 %)	 BMW
[--------------------]: (0.056 %)	 K3
[--------------------]: (0.091 %)	 K5
[--------------------]: (0.090 %)	 K7
[--------------------]: (0.038 %)	 QM3
[--------------------]: (0.055 %)	 그랜저
[--------------------]: (0.102 %)	 벤츠E
[--------------------]: (0.056 %)	 산타페
[--------------------]: (0.086 %)	 소나타
[--------------------]: (0.023 %)	 스포티지
[--------------------]: (0.010 %)	 싼타페
[--------------------]: (0.031 %)	 쏘나타
[--------------------]: (0.018 %)	 쏘렌토
[--------------------]: (0.076 %)	 아반떼
[--------------------]: (0.004 %)	 아반테
[--------------------]: (0.073 %)	 제네시스
[--------------------]: (0.006 %)	 코란도C
[--------------------]: (0.011 %)	 투싼
[--------------------]: (0.036 %)	 티구안
[####################]: (3.255 %)	 티볼리
[--------------------]: (0.030 %)	 파사트
[#-------------------]: (0.246 %)	 폭스바겐골프
[--------------------]: (0.067 %)	 현기차
[-------